## Steps to run the API Locally using the Flask Server (No Containerization)

The following steps describe how to run the Flask API locally with the standard Flask server, so that you can test endpoints before you containerize the app:

Install python dependencies - These dependencies are kept in a requirements.txt file. To install them, go to the project directory that you’ve just downloaded, and use pip as follows:

 >`pip install -r requirements.txt`

Set up the environment - You do not need to create an .env_file to run locally but you do need the following two variables available in your terminal environment:
* JWT_SECRET - The secret used to make the JWT, for the purpose of this course the secret can be any string.
* LOG_LEVEL - It represents the level of logging. It is optional to be set. It has a default value as 'INFO', but when debugging an app locally, you may want to set it to 'DEBUG'. 

To add these to your terminal environment, run the following:

>```
export JWT_SECRET='myjwtsecret'
export LOG_LEVEL=DEBUG```

Run the app using the Flask server, from the top directory, run:

 >`python main.py`

Open `http://127.0.0.1:8080/` in a new browser OR run 
>`curl --request GET http://localhost:8080/`
on the command-line terminal. It will give you a response as `"Healthy"`.

Try the API endpoints on Command-Line
Open another terminal window, and install jq, which is a package that helps to read or manipulate JSON processors. 
For Linux,

>`sudo apt-get install jq`

For Mac,

>`brew install jq`

For Windows,

>`chocolatey install jq`

For other OS, refer [here](https://stedolan.github.io/jq/download/)

To try the `/auth` endpoint, use the following command, replacing email/password as applicable to you:

>```
export TOKEN=`curl --data '{"email":"abc@xyz.com","password":"mypwd"}' --header "Content-Type: application/json" -X POST localhost:8080/auth  | jq -r '.token'`
```

This calls the endpoint 'localhost:8080/auth' with the email/password as the message body. The return value is a JWT token based on the secret string you supplied. We are assigning that secret to the environment variable 'TOKEN'. To see the JWT token, run:

>`echo $TOKEN`

To try the /contents endpoint which decrypts the token and returns its content, run:

 >```curl --request GET 'http://localhost:8080/contents' -H "Authorization: Bearer ${TOKEN}" | jq .```
 
You should see the email id that you passed in as one of the values.

## Containerize the Flask App and Run Locally
The following steps describe how to complete the Dockerization part of the project. After you complete these steps, you should have the Flask application up and running in a Docker container.

If you haven't installed Docker already, you should install now using these installation instructions. Start the Docker Desktop services. You would see the Docker “whale” icon in your status bar. In trouble, refer to the Docker Desktop documentation, as appropriate for your underlying OS.

Create a Dockerfile named Dockerfile in the app repo. Your Dockerfile should:

Use the `python:stretch` image as a source image

Set up an `app` directory for your code

`Install pip` and needed Python packages from `requirements.txt`

Define an entrypoint which will run the main app using the Gunicorn WSGI server. The Gunicorn should run with the arguments as follows: 

>```["gunicorn", "-b", ":8080", "main:APP"].```

Create a file named `.env_file` and save both `JWT_SECRET` and `LOG_LEVEL` into `.env_file`. These environment variables will run locally in your container. Here, we do not need the `export` command, just an equals sign:

>```
JWT_SECRET='myjwtsecret'
 LOG_LEVEL=DEBUG```

This `.env_file` is only for the purposes of running the container locally, you do not want to check it into github or other public repositories.
 You can prevent this by adding it to your `.gitignore` file, which will cause git to ignore it. To safely store and use secrets in the cloud, use a secure solution such as AWS’s parameter store.

Build a local Docker image with the tag `jwt-api-test`
>```
docker build -t myimage .
# Check the list of images
docker image ls
# Remove any image
docker image rm <image_id>```

Create and run a container using the image locally:
* You can pass the name of the env file using the flag `--env-file=<YOUR_ENV_FILENAME>`.

* You should expose the port 8080 of the container to the port 80 on your host machine.

>```
docker run --name myContainer --env-file=.env_file -p 80:8080 myimage
# List running containers
docker container ls
docker ps
# Stop a container
docker container stop <container_id>
# Remove a container
docker container rm <container_id>
```

To use the endpoints, you can use the same curl commands as before, except using port 80 this time. Open a new terminal window, and try the following command:

>```curl --request GET 'http://localhost:80/'```

or check http://localhost:80/ in the browser. You should see a "Healthy" response.
>```Known Issue: You may not be able to run the /auth endpoint due to locale issues the Python container. It can be resolved by adding a few more environments in the container, as discussed in this [thread]() or [return a simple string from the auth() method in main.py](). Although, this step is not required for the project.```